In [7]:
import pandas as pd
import numpy as np

In [26]:
result_cot_general = pd.read_csv('experiment_results/gpt-turbo_multiarith_cot_general_8shot.csv', index_col=0)

result_arith_general = pd.read_csv('experiment_results/gpt-turbo_multiarith_arithcot_general_8shot.csv', index_col=0)
result_arith_multi = pd.read_csv('experiment_results/gpt-turbo_multiarith_arithcot_multiarith_8shot.csv', index_col=0)

result_arith_general_old = pd.read_csv('experiment_results/old/gpt-turbo_multiarith_arithcot_general_8shot.csv', index_col=0)
result_arith_multi_old = pd.read_csv('experiment_results/old/gpt-turbo_multiarith_arithcot_multiarith_8shot.csv', index_col=0)

In [27]:
def acc(results):
    return np.mean(results['answer'] == results['response_answer'])

def missing(results):
    np.sum(results['response_answer'].isnull())

In [28]:
print(acc(result_cot_general))
print(acc(result_arith_general))
print(acc(result_arith_multi))
print(acc(result_arith_general_old))
print(acc(result_arith_multi_old))

0.975
0.8983333333333333
0.965
0.9116666666666666
0.965


In [29]:
print(np.sum(result_arith_general['response_answer'].isnull()))
print(np.sum(result_arith_multi['response_answer'].isnull()))
print(np.sum(result_arith_general_old['response_answer'].isnull()))
print(np.sum(result_arith_multi_old['response_answer'].isnull()))

39
15
33
6


## Two different prompts

In [38]:
result_arith_general_old.columns = [i + "_old" for i in result_arith_general_old.columns]

In [39]:
c_arith_general = pd.concat([result_arith_general, result_arith_general_old], axis = 1)

In [42]:
c_arith_general_errors = c_arith_general[['answer','response_answer','response_answer_old']]

In [46]:
c_arith_general[c_arith_general.response_answer != c_arith_general.response_answer_old].count()

question               87
answer                 87
response               87
mapping                87
response_answer        48
equations              87
question_old           87
answer_old             87
response_old           87
mapping_old            87
response_answer_old    54
equations_old          87
dtype: int64

In [51]:
result_arith_general_old[result_arith_general_old['response_answer_old'].isnull()].to_csv('temp.csv')